<a href="https://colab.research.google.com/github/cellatlas/human/blob/master/markers/eye/eye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gget
!pip install -q git+https://github.com/sbooeshaghi/ec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:

import pandas as pd
import numpy as np
from ec.utils import write_markers

In [3]:
# Get valid gene names
!wget -O genes.txt https://caltech.box.com/shared/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
genes_list = pd.read_csv('genes.txt', header = None)[0].values

--2023-03-19 23:27:15--  https://caltech.box.com/shared/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
Resolving caltech.box.com (caltech.box.com)... 74.112.186.144
Connecting to caltech.box.com (caltech.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt [following]
--2023-03-19 23:27:15--  https://caltech.box.com/public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
Reusing existing connection to caltech.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://caltech.app.box.com/public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt [following]
--2023-03-19 23:27:15--  https://caltech.app.box.com/public/static/4r5prhstoq2j1lk8l56bpwqqvfj54q2u.txt
Resolving caltech.app.box.com (caltech.app.box.com)... 74.112.186.144
Connecting to caltech.app.box.com (caltech.app.box.com)|74.112.186.144|:443... connected.
HTTP request sent, awaiting response... 3

# Eye

In [4]:
species = "homo_sapiens"
organ = "eye"
reference = "hg38"
paper_doi = "https://dx.doi.org/10.1038%2Fs41467-021-25968-8"
table_link = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8478974/bin/41467_2021_25968_MOESM5_ESM.xlsx"

# don't include in header
table_name = "41467_2021_25968_MOESM5_ESM.xlsx"

header = [
    {
      "species": species,
      "organ": organ,
      "reference": reference,
      "paper_doi": paper_doi,
      "table_link": table_link,
    }
]

In [5]:
excel_1 = pd.read_excel(table_link, skiprows = 1, sheet_name = "ChoroidSclera")
excel_2 = pd.read_excel(table_link, skiprows = 1, sheet_name = "Cornea")
excel_3 = pd.read_excel(table_link, skiprows = 1, sheet_name = "IrisCiliaryBody")


df_1 = excel_1.rename(columns={"cluster": "celltype"})
df_2 = excel_2.rename(columns={"cluster": "celltype"})
df_3 = excel_3.rename(columns={"cluster": "celltype"})

# Decided to unify the cell types from the 3 tissues within the eye because otherwise many genes were repeated.
# df_1["celltype"] = "ChoroidSclera_" + df_1["celltype"]
# df_2["celltype"] = "Cornea_" + df_2["celltype"]
# df_3["celltype"] = "IrisCiliaryBody" + df_3["celltype"]

df = pd.concat([df_1, df_2, df_3])


In [6]:
bidx = df['gene'].isin(genes_list)
print(f'Filtered {np.sum(~bidx)} out of {len(bidx)} genes')
df = df[bidx]

Filtered 900 out of 15538 genes


In [7]:
df.head()

,Unnamed: 0,p_val,avg_logFC,pct.1,pct.2,p_val_adj,celltype,gene
0,IGFBP5,1.917143e-260,2.740738,0.981,0.809,3.479997e-256,Fibroblasts,IGFBP5
1,MGP,8.043193e-214,1.760688,1.000,0.992,1.460000e-209,Fibroblasts,MGP
2,RARRES1,2.216642e-198,2.315193,0.905,0.622,4.023649e-194,Fibroblasts,RARRES1
3,FBLN1,4.029294e-198,1.967178,0.894,0.596,7.313975e-194,Fibroblasts,FBLN1
4,C1R,3.932635e-193,1.828830,0.872,0.491,7.138518e-189,Fibroblasts,C1R


In [15]:
min_mean = 100
max_pval = 1e-10
min_lfc = 1.5
max_gene_shares = 2
max_per_celltype = 20

# filter by criteria
dfc = df.query(f"p_val_adj <= {max_pval} & avg_logFC >= {min_lfc}")

# mask out genes that are shared between max_gene_shares cell type
non_repeat_genes = dfc["gene"].value_counts()[dfc["gene"].value_counts() < max_gene_shares].index.values

m = dfc[dfc.gene.isin(non_repeat_genes)].sort_values('pct.1', ascending = True)

# max number to sample is equal to the min number of genes across all celltype
n_sample = min(m["celltype"].value_counts().min(), max_per_celltype)

# sample n_sample genes
markers = m.groupby('celltype').tail(10)
markers_dict = markers.groupby("celltype")["gene"].apply(lambda x: list(x)).to_dict()

In [16]:
m.celltype.value_counts()

Monocytes                            52
Melanocytes                          22
Fibroblasts                          20
Schwann cells                        17
Smooth muscle cells                  15
Activated T cells                    14
Ciliary body endothelial cells       13
Putative stem cells                   9
Cytotoxic T cells                     5
Conjunctival cells                    5
Choroid endothelial cells             4
Ciliary body cells                    2
TGFBI-hi corneal epithelial cells     1
Name: celltype, dtype: int64

In [17]:
markers.groupby("celltype")["pct.1"].mean().sort_values()


celltype
Choroid endothelial cells            0.7655
Activated T cells                    0.8042
Schwann cells                        0.8105
Cytotoxic T cells                    0.8674
Ciliary body cells                   0.8780
Fibroblasts                          0.9298
Ciliary body endothelial cells       0.9368
Conjunctival cells                   0.9408
Putative stem cells                  0.9500
Melanocytes                          0.9909
Smooth muscle cells                  0.9993
Monocytes                            1.0000
TGFBI-hi corneal epithelial cells    1.0000
Name: pct.1, dtype: float64

In [18]:
write_markers("markers.txt", markers_dict, header)

In [19]:
!cat markers.txt

# homo_sapiens	eye	hg38	https://dx.doi.org/10.1038%2Fs41467-021-25968-8	https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8478974/bin/41467_2021_25968_MOESM5_ESM.xlsx
Activated T cells	STAT4,HIST1H1C,CST7,CD3E,ISG20,STK4,CLEC2B,ACAP1,ZFP36L2,HLA-C
Choroid endothelial cells	HYAL2,MPZL2,PRSS23,PDLIM1
Ciliary body cells	CRYAA,CXCL14
Ciliary body endothelial cells	SPRY1,SOCS2,GIMAP7,A2M,TGM2,S1PR1,KCTD12,TMSB10,SPARCL1,HSPB1
Conjunctival cells	AQP5,KRT4,SLPI,KRT13,IL1RN
Cytotoxic T cells	IRF8,CD37,MS4A1,CD79A,LINC00926
Fibroblasts	MEG3,C1S,FBLN1,RARRES1,PLA2G2A,GSN,PTGDS,MT2A,APOD,MGP
Melanocytes	CYB561A3,CDC42EP3,PHLDA1,GPX3,APOE,VIM,CD63,CD59,UBB,KLF6
Monocytes	RNASET2,CST3,C15orf48,CTSZ,COTL1,INSIG1,CREM,HLA-DRB5,CYBA,HSPD1
Putative stem cells	ATP6V1C2,GJA1,SLC35G1,SDS,GNG11,DAPL1,FRZB,TFPI2,CLU
Schwann cells	CHL1,ERBB3,SCN7A,CDH19,SPARC,PLP1,S100B,ABCA8,CD9,LGI4
Smooth muscle cells	ATP2A2,ACTA2,PPP1R12A,MYL9,CKB,MYH11,DSTN,TPM2,MYLK,TPM1
TGFBI-hi corneal epithelial cells	ALDH3A1


In [ ]:
# Download table to have a local copy
!wget $table_link -O deg.xlsx

--2023-03-17 17:44:00--  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8478974/bin/41467_2021_25968_MOESM5_ESM.xlsx
Resolving www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)... 130.14.29.110, 2607:f220:41e:4290::110
Connecting to www.ncbi.nlm.nih.gov (www.ncbi.nlm.nih.gov)|130.14.29.110|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1272133 (1.2M) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘deg.xlsx’

deg.xlsx            100%[===================>]   1.21M  3.44MB/s    in 0.4s    

2023-03-17 17:44:01 (3.44 MB/s) - ‘deg.xlsx’ saved [1272133/1272133]

